In [1]:
import os

os.chdir("../")  # Укажите путь к корню проекта
print(os.getcwd())  # Проверьте, что путь изменился

c:\Users\MishaV\Desktop\karpov\ML\L22


In [2]:
import warnings
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
from src.data import (
    select_data_csv,
    CustomTransformer,
    train_test_split,
    transform_data,
    OHE_COLS,
    MTE_COLS,
)
from src.train import save_model, load_model, model_score

warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [4]:
# data = select_data(
#     seed=0.42,
#     count=4_800_000,
#     method="stratified",
#     users_scv=True,
#     posts_csv=True,
#     feed_data_csv=True,
#     save_path=r".\data\raw\stratified42.csv",
# )
data = pd.read_csv(r".\data\raw\stratified42.csv", parse_dates=["timestamp"])
data["timestamp"].is_monotonic_increasing

True

In [5]:
print("target с значением 1: ", (data["target"] == 1).sum())
print("target с значением 0: ", (data["target"] == 0).sum())

target с значением 1:  589259
target с значением 0:  4369645


In [6]:
X = data.drop("target", axis=1)
y = data["target"]

# более новые данные (по timestamp) относим к тестовой выборке
X_train, X_test, y_train, y_test = train_test_split(X, y, 0.2)
custom_cv = [(X_train.index, X_test.index)]

CatBoost

In [7]:
pipe = Pipeline(
    [
        ("custom_transformer", CustomTransformer()),
        (
            "model",
            CatBoostClassifier(
                random_state=42,
                auto_class_weights="Balanced",
                cat_features=OHE_COLS + MTE_COLS,
            ),
        ),
    ]
)

In [8]:
pipe.fit(X_train, y_train)

Learning rate set to 0.354391
0:	learn: 0.6704181	total: 2.24s	remaining: 37m 21s
1:	learn: 0.6622273	total: 4.18s	remaining: 34m 44s
2:	learn: 0.6583246	total: 6.21s	remaining: 34m 22s
3:	learn: 0.6555761	total: 8.07s	remaining: 33m 30s
4:	learn: 0.6489351	total: 9.71s	remaining: 32m 12s
5:	learn: 0.6484725	total: 11.2s	remaining: 30m 57s
6:	learn: 0.6474253	total: 13.3s	remaining: 31m 33s
7:	learn: 0.6472737	total: 14.8s	remaining: 30m 33s
8:	learn: 0.6472452	total: 15.7s	remaining: 28m 45s
9:	learn: 0.6466436	total: 17.6s	remaining: 28m 58s
10:	learn: 0.6462131	total: 19.1s	remaining: 28m 38s
11:	learn: 0.6459574	total: 20.9s	remaining: 28m 43s
12:	learn: 0.6438490	total: 23.1s	remaining: 29m 10s
13:	learn: 0.6425747	total: 25s	remaining: 29m 18s
14:	learn: 0.6422578	total: 27s	remaining: 29m 32s
15:	learn: 0.6419866	total: 29s	remaining: 29m 44s
16:	learn: 0.6415155	total: 30.8s	remaining: 29m 40s
17:	learn: 0.6410954	total: 32.4s	remaining: 29m 25s
18:	learn: 0.6410755	total: 33.2

Pipeline(steps=[('custom_transformer',
                 CustomTransformer(drop_cols=['text', 'user_id', 'post_id', 'timestamp', 'country', 'action'])),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x000002308FB64B50>)])

In [9]:
y_proba = pipe.predict_proba(X_test)[:, 1]

In [10]:
model_score(X_test, y_test, y_proba)

roc_auc_score: 0.4452519696313292
average_precision_score: 0.10199963294969959
hitrate: 0.461959933404296


In [11]:
save_model("catboost", pipe.named_steps["model"], method="catboost")

In [13]:
model = load_model("catboost250115141627.cbm", method="catboost")

In [14]:
pipe.set_params(model=model)

Pipeline(steps=[('custom_transformer',
                 CustomTransformer(drop_cols=['text', 'user_id', 'post_id', 'timestamp', 'country', 'action'])),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x000002307E411490>)])

In [15]:
y_proba = pipe.predict_proba(X_test)[:, 1]

In [16]:
model_score(X_test, y_test, y_proba)

roc_auc_score: 0.4452519696313292
average_precision_score: 0.10199963294969959
hitrate: 0.461959933404296
